## 1. Load data from all sessions

#### Load name and path of all sessions

In [4]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from session_utils import find_all_sessions

# Find all included sessions from Google sheet, with structure session_name: path
session_dict = find_all_sessions(sheet_path = 'https://docs.google.com/spreadsheets/d/1_Xs5i-rHNTywV-WuQ8-TZliSjTxQCCqGWOD2AL_LIq0/edit#gid=0',
                                 data_path = '/home/isabella/Documents/isabella/jake/recording_data',
                                 sorting_suffix = 'sorting_ks2_custom')
print(f'{len(session_dict.items())} sessions found')

57 sessions found


## 2. Loop over all sessions, load data and filter for pyramidal cells
### Criteria:
#### - A. Cluster marked 'good' in phy
#### - B. Cluster depth 0 +-200um
#### - C. Mean FR < 10 Hz
#### - D. Mean spike width >300us
#### - E. Burst index - first moment of AC < 25

In [5]:
from ephys import *
from scipy.signal import argrelmax

for session, session_path in session_dict.items():
    
    # Create ephys object
    obj = ephys(recording_type = 'nexus', path = session_path)
    
    ## A. Load good cells from phy
    obj.load_spikes('good')
    
    # Get cluster info from phy
    cluster_info = obj.spike_data['cluster_info']
    # Get total good cells for session
    total_cells = len(cluster_info.index)
    
    
    ## B. Get cluster depths and exclude any outside of 0 +-200um
    cluster_info = cluster_info[cluster_info['depth'].between(-200, 200)].copy()
    

    ## C. Filter for mean firing rate between 0-10 Hz
    cluster_info = cluster_info[cluster_info['fr'].between(0, 10)]
    
    ## D. Filter for spike width >300 us from template
    if not cluster_info.empty:
        obj.load_mean_waveforms(clusters_to_load = list(cluster_info.index)) #Pick up to 500 spikes at random for performance

        # Calculate spike width for each cluster from mean of every 50th spike 
        for cluster, mean_waveform in obj.mean_waveforms.items():
            
            # Find peak to trough time in us
            peak = np.argmin(mean_waveform)

            trough = argrelmax(mean_waveform[peak:])[0] #Next local maxmimum after (negative) peak
            if len(trough) > 1:
                trough = trough[0]

            peak_to_trough = trough - peak
            
            cluster_info.loc[cluster, 'spike_width_microseconds'] = (peak_to_trough / obj.spike_data['sampling_rate']) * 1e6
        
        # Filter for spike width > 300us as in Wills et al., 2010
        cluster_info = cluster_info[cluster_info['spike_width_microseconds'] > 300].copy()

    
    ## E. Calculate burst index and filter
    if not cluster_info.empty:
        
        # Reload spike data only for included cells
        clusters_inc = list(cluster_info.index)
        obj.load_spikes(clusters_to_load = clusters_inc)
        
        # Generate autocorrelograms and burst index for each cluster
        from burst_index_and_autocorrelograms import *

        spike_times_inc = obj.spike_data['spike_times']
        spike_clusters_inc = obj.spike_data['spike_clusters']

        autocorrelograms, first_moments = compute_autocorrelograms_and_first_moment(spike_times_inc, 
                                                                                     spike_clusters_inc, 
                                                                                     bin_size = 0.001, #1ms
                                                                                     time_window = 0.05) #50ms
        
        cluster_info['first_moment_AC'] = first_moments.values()
        
        # Filter for first moment <25
        cluster_info = cluster_info[cluster_info['first_moment_AC'] < 25]
            
    ## SAVE INCLUDED CLUSTER IDs TO .NPY
    clusters_inc = cluster_info.index
    n_clusters_inc = len(cluster_info.index)

    np.save(f'{session_path}/clusters_inc.npy', clusters_inc)        
        
    print(f'Session {session}: {n_clusters_inc} cells retained of {total_cells} good cells from phy. Retained cells: {clusters_inc.values}')

[-583.33333333]
Session 230503_r1354: 0 cells retained of 5 good cells from phy. Retained cells: []
[-458.33333333]
[-500.]
[-458.33333333]
Session 230504_r1354: 0 cells retained of 8 good cells from phy. Retained cells: []
Session 230505_r1354: 0 cells retained of 4 good cells from phy. Retained cells: []
-541.6666666666666
-562.5
[-500.]
[-520.83333333]
[-270.83333333]
[-145.83333333]
[-166.66666667]
[-208.33333333]
[-375.]
[-520.83333333]
[-250.]
[-41.66666667]
[-83.33333333]
[-83.33333333]
[-104.16666667]


ValueError: Must have equal len keys and value when setting with an iterable

In [1]:
cluster_info


NameError: name 'cluster_info' is not defined